In [ ]:
%matplotlib inline

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# PHYS 395 - week 3

**Matt Wiens - #301294492**

This notebook will be organized similarly to the lab script, with major headings corresponding to the headings on the lab script.

*The TA's name (Ignacio) will be shortened to "IC" whenever used.*

## Setup

In [ ]:
# Set default plot size
plt.rcParams["figure.figsize"] = (10, 7)

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999

# Molecular dynamics

We're going to use some principle from molecular dynamics (MD), but instead of simulating many particles we're going to keep it simple and only simulate a few.

## Verlet algorithm

Given initial data $r_i(0), v_i(0), a_i(0)$, using the Verlet algorithm we can calculate the first update in position using

\begin{equation}
    r_i(\Delta t) = r_i(0) + v_i(0) \Delta t + \frac{1}{2} a_i(0) \Delta t^2
    ,
\end{equation}

and subsequent changes in position using

\begin{equation}
    r_i(t + \Delta t) = 2 r_i(t) - r_i(t - \Delta t) + a_i(t) \Delta t^2
    ,
\end{equation}

where $r_i, v_i, a_i$ are understood to be vector quantities.

The first equation above should be self-explanatory. The second equation we can derive by making use of Taylor series:

\begin{equation}
    r_i(t \pm \Delta t)
        = r_i(t)
            \pm r_i^\prime(t) \Delta t
            + \frac{1}{2} r_i^{\prime\prime}(t) \Delta t ^2
            \pm \frac{1}{6} r_i^{\prime\prime\prime}(t) \Delta t^3
            + \mathcal{O}(\Delta t^4)
            .
\end{equation}

Thus

\begin{align}
    r_i(t + \Delta t)
        &= r_i(t)
            + r_i^\prime(t) \Delta t
            + \frac{1}{2} r_i^{\prime\prime}(t) \Delta t ^2
            + \frac{1}{6} r_i^{\prime\prime\prime}(t) \Delta t^3
            + \mathcal{O}(\Delta t^4) \\
        &= 2 r_i(t)
            + r_i^{\prime\prime}(t) \Delta t ^2
            - \left(
                r_i(t)
                - r_i^\prime(t) \Delta t
                + \frac{1}{2} r_i^{\prime\prime}(t) \Delta t ^2
                - \frac{1}{6} r_i^{\prime\prime\prime}(t) \Delta t^3
            \right)
            + \mathcal{O}(\Delta t^4) \\
        &= 2 r_i(t)
            + r_i^{\prime\prime}(t) \Delta t ^2
            - r_i(t - \Delta t)
            + \mathcal{O}(\Delta t^4) \\          
        &= 2 r_i(t)
            - r_i(t - \Delta t)
            + a_i(t) \Delta t ^2
            + \mathcal{O}(\Delta t^4)
        .
\end{align}

As we can see, this is good to $\mathcal{O}(\Delta t^4)$. (Note that the above derivation is equivalent to one using finite-difference methods, since those are based on Taylor series anyway.)

Let's write two functions that implement the Verlet algorithm functions.

In [ ]:
def verlet_1(r: np.ndarray, v: np.ndarray, a: np.ndarray, dt: float) -> np.ndarray:
    """Implements the first Verlet algorithm equation."""
    return r + dt * v + dt ** 2 / 2 * a

def verlet_2(r_1: np.ndarray, r_2: np.ndarray, a: np.ndarray, dt: float) -> np.ndarray:
    """Implements the second Verlet algorithm equation.
    
    Note that the input parameter r_1 specifies the position at time t,
    while the input parameter r_2 specifies the position at time t - dt.
    """
    return 2 * r_1 - r_2 + dt ** 2 * a

Let's test to make sure our functions do what they should do.

In [ ]:
# Set up initial data
init_pos = np.array([0, 3])
init_vel = np.array([1, 5])

# These are constant
accel = np.array([0, -9.8])
dt = 0.1

In [ ]:
# Test
num_iters = 20

t = 0
last_pos = init_pos
pos = init_pos

print_test = lambda r, t: print("t=%.2f\tx=%.2f\ty=%.2f" % (t, r[0], r[1]))

# 0th iter
print_test(pos, t)

# 1st iter
pos = verlet_1(pos, init_vel, accel, dt)
t += dt

print_test(pos, t)

# Remaining iters
for _ in range(num_iters - 1):
    pos, last_pos = (verlet_2(pos, last_pos, accel, dt), pos)
    t += dt
    
    print_test(pos, t)

# Flight to the Moon: Apollo 8

Now we're going to try to build a simulation around launching a rocket from Earth, with the goal of orbiting around the Moon and returning safely back to Earth.

## Setup

First let's set up the physical constants we'll need.

In [ ]:
# Gravity constant in N m^2 / kg^2
CONST_G = 6.6743015e-11

# Mass of earth and moon in kg
MASS_EARTH = 5.972e24
MASS_MOON = 7.34767e22

# Radius measurements in m
RADIUS_MOON_EARTH_ORBIT = 3.84e8
RADIUS_EARTH = 6.3781e6
RADIUS_MOON = 1.74e6

# Period of Moon's orbit around earth
PERIOD_MOON_EARTH_ORBIT = math.sqrt(
    4 * np.pi ** 2 * RADIUS_MOON_EARTH_ORBIT ** 3 / (CONST_G * MASS_EARTH)
)
ANGULAR_FREQ_MOON_EARTH_ORBIT = math.sqrt(
    CONST_G * MASS_EARTH / RADIUS_MOON_EARTH_ORBIT ** 3
)

In [ ]:
# Use a class as a namespace to abbreviate the constants without
# polluting main namespace
class Attrs:
    pass


consts = Attrs()
consts.G = CONST_G
consts.ME = MASS_EARTH
consts.MM = MASS_MOON
consts.D = RADIUS_MOON_EARTH_ORBIT
consts.RE = RADIUS_EARTH
consts.RM = RADIUS_MOON
consts.T = PERIOD_MOON_EARTH_ORBIT
consts.omega = ANGULAR_FREQ_MOON_EARTH_ORBIT

Note that in the Verlet algorithm

\begin{equation}
    a_i(t) = \frac{F_i(t)}{m_i}
\end{equation}

has $m_i$ in the denominator, and the gravity force has $m_i$ in the numerator. Both masses cancel each other out, and hence we don't need to worry about the mass of the rocket when doing our simulation. This is because we're going to assume that the force exerted on the Earth/Moon by the rocket is negligibly small.

The values of the period of the Moon's orbit around earth $T$ and the corresponding angular frequency $\omega$ are shown below.

In [ ]:
print("T\t= %e s" % consts.T)
print("omega\t= %e rad/s" % consts.omega)

## Estimating required launch speed

In our simulation, we're going to exclusively consider three particles: the Earth, the Moon, and the rocket. The Earth will be fixed at the origin of our coordinate system, and the Moon will orbit around the Earth in a fixed trajectory.

Using conservation of energy, we can estimate the required launch velocity $v_0$ using

\begin{align}
    &\frac{1}{2} m v_0^2 - \frac{G M_E m}{R_E} = - \frac{G M_E m}{D} \\
    &\Rightarrow v_0 = \sqrt{ 2 G M_E \left( \frac{1}{R_E} - \frac{1}{D} \right) }
    .
\end{align}

We estimate $v_0$ using this formula below.

In [ ]:
approx_v_0 = math.sqrt(2 * consts.G * consts.ME * (1 / consts.RE - (1 / consts.D)))

print("initial required velocity estimate: %.2f m/s" % approx_v_0)

## Computing the rocket's acceleration

Let's write a function that gives us the acceleration of the rocket given the position of the rocket, the Moon, and the Earth. Since the Earth is fixed at the origin, we don't need to explicitly pass it in.

In [ ]:
def rocket_accel(r_ship: np.ndarray, r_moon: np.ndarray) -> np.ndarray:
    """Find the acceleration on the rocket due to the Moon and Earth."""
    # Calculate displacement vectors
    delta_r_moon = r_moon - r_ship
    delta_r_earth = r_ship

    # Calculate acceleration due to Moon and Earth
    a_moon = -delta_r_moon * consts.G * consts.MM / np.linalg.norm(delta_r_moon) ** 3
    a_earth = -delta_r_earth * consts.G * consts.ME / np.linalg.norm(delta_r_earth) ** 3

    # Return the total acceleration
    return a_moon + a_earth

Let's test this acceleration function by seeing what it returns when the rocket is at position $(R_E, 0)$ and the Moon is at position $(D, 0)$.

In [ ]:
res = rocket_accel(np.array([consts.RE, 0]), np.array([consts.D, 0]))

print("test acceleration: %s" % res)

Looks good.

## Simulation

Now let's actually simulate the rocket. We'll try to find an appropriate $v_0$ that results in a successful trajectory given an $\alpha$ which tells us the initial angle of the Moon.